In [ ]:
# transform data into csv for ie test

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
data_path = "./SICK_trial_and_train.txt"

In [6]:
def sick2csv(path):
    data = []
    with open(data_path, "r") as f:
        next(f)
        for line in f.readlines():
            a = line.split("\t")
            ph = a[1].split(" [SEP] ")
            p = ph[0]
            h = ph[1]
            l = a[2][0]
            l = l.replace('0', 'entailment')
            l = l.replace('1', 'contradiction')
            l = l.replace('2', 'neutral')
            data.append((p, h, l))
        data = data[1:]
        columns = ("premise", "hypothesis", "label")
    return pd.DataFrame(data, columns=columns)

In [7]:
df_data = sick2csv(data_path)
# df_data.to_csv("./train.csv", index=False)

In [8]:
# split data
train, test = train_test_split(df_data, random_state=2018, test_size=0.1, shuffle= True)

In [11]:
train.to_csv("./train.csv", index=False)
test.to_csv("./test.csv", index=False)

In [10]:
test

,premise,hypothesis,label
3158,A girl is skateboarding on a half pipe.,A skateboarder is doing tricks on a half pipe.,neutral
2979,A man is playing guitar.,A woman is playing the drums.,neutral
2323,The log is lying on the panda bear.,The panda bear is lying on the logs.,contradiction
4937,A large dog and a small dog are standing next ...,A smiling boy in winter clothes is waving his ...,neutral
673,Some people are walking.,A group of scouts are camping in the grass.,neutral
...,...,...,...
1024,A man is speaking on a podium.,A podium is where a man is speaking.,entailment
2423,A woman is picking a can.,There is no woman slicing a tomato.,neutral
3367,The white dog wearing reindeer ears is close t...,The white dog wearing reindeer ears is near a ...,entailment
2634,A dog is being teased by a monkey at the zoo.,A monkey is pulling a dog's tail.,neutral


In [ ]:
# check if transformation worked

In [6]:
columns = ["premise", "hypothesis", "label"]

In [5]:
df_train = pd.read_csv('./train_clean.csv')

In [65]:
def clean_trans(df):
    for index, row in df.iterrows():
        row['hypothesis'] = row['hypothesis'].strip()
    return df

In [83]:
df_train_rho20_0 = clean_trans(df_train_rho20_0)

In [57]:
df_train_rho20_0 = pd.read_csv('./rho20/train_rho20-0.csv')

In [58]:
df_train_rho20_0 = df_train_rho20_0[columns]

In [80]:
pi = []
hi = []
ps = []
hs = []

for i in range(len(df_train)):
    p_base = df_train.iloc[i]['premise']
    h_base = df_train.iloc[i]['hypothesis']
    p_trans = df_train_rho20_0.iloc[i]['premise']
    h_trans = df_train_rho20_0.iloc[i]['hypothesis']
    if p_base != p_trans:
        pi.append(i)
        ps.append((p_base, p_trans))
    
    if h_base != h_trans:
        hi.append(i)
        hs.append((h_base, h_trans))

In [ ]:
# Convert to hy-nli sick data original format

In [7]:
df_train

,premise,hypothesis,label
0,men are sawing logs,men are cutting wood,entailment
1,a man is passionately playing a guitar,a person is singing and playing a guitar,neutral
2,a man with no shirt is jumping over a log,a guy is leaping into the air in a wooded area,neutral
3,there is no one sitting in lawn chairs and rea...,two people in two chairs are reading,neutral
4,a man is cutting a potato,a man is cutting a tomato,neutral
...,...,...,...
4494,a cat is playing a piano,a cat is sitting on a piano,neutral
4495,two toddlers are eating corndogs in a wagon wh...,two young kids are eating corndogs,entailment
4496,some sticky goo is being pulled out of a bowl ...,a woman is pulling some sticky goo out of a bowl,entailment
4497,a woman is cutting potatoes,a woman is slicing potatoes,entailment


In [ ]:
# cols = ['pair_ID', 'sentence_AB', 'E_judgment']

In [8]:
output_path = './rho0/train_rho0-0-final.csv'

In [13]:
def csv2sick(df, output_path):
    sep = '[SEP]'
    tab = '\t'
    with open(output_path, "w") as f:

        heading = 'pair_ID' + tab + 'sentence_AB' + tab + 'E_judgment'
        f.write(heading)

        for index, row in df.iterrows():
            p = row['premise']
            h = row['hypothesis']
            l = row['label']

            l = l.replace('entailment', '0')
            l = l.replace('contradiction', '1')
            l = l.replace('neutral', '2')

            sentence_ab = p + ' ' + sep + ' ' + h
            line = '\n' + str(index) + tab + sentence_ab + tab + l
            f.write(line)


In [14]:
rho_base = 'rho'
csv_ext = '.csv'

for r in range(0,120,20):
    for i in range(5):
        rho = rho_base + str(r)
        path_base = './' + rho + '/' + 'train_' + rho + '-' + str(i)
        input_path = path_base + csv_ext
        output_path = path_base + '-final' + csv_ext
        df = pd.read_csv(input_path)
        csv2sick(df, output_path)    